<a href="https://colab.research.google.com/github/luqmanrofifm/Excel_Table_Automation/blob/main/ExcelAutomation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops
from itertools import combinations 

In [45]:
def cleaningTable(data_path):

    xl = pd.ExcelFile(data_path)
    res = len(xl.sheet_names)

    list_sheet = []

    #=========read excel file
    for i in xl.sheet_names:
          df = pd.read_excel(data_path, sheet_name=i, engine='openpyxl')
          list_sheet.append(df)

    #=========remove sheet kosong
    list_clean_sheet = []
    for i in list_sheet:
      size = i.shape
      if ((size[0] == 0) and (size[1] == 0)):
        continue
      else:
        list_clean_sheet.append(i)

    #========parsing table
    list_data = []
    for i in list_clean_sheet:
      list_data.append(parseTabel(i))

    #========remove None Row in table
    for i in list_data:  
      for index,y in enumerate(i):
        i[index] = removeNoneRow(y)
  

    #========delete those that are not tables
    #counter1 = 0
    #counter2 = 0
    list_clean_parsing = []
    for i in list_data:
      #print(counter1)
      temporary = []
      for y in i :
        #print(counter2)
        check = isItTable(y)
        #print(check)
        if (check):
          temporary.append(y)
          #continue
        else:
          continue
      list_clean_parsing.append(temporary)

    #========check duplicate table
    #for i in list_data:
    #  comb = combinations(i,2)
    #  for y in comb:
    #    return_similarity = similarityTable(y[0],y[1]) 
    #    if (return_similarity == 1):
    #      continue
    #    elif (return_similarity == 2):
    #      return_data_similarity = checkSimilarityDataTable(y[1],y[0])
    #      if (return_data_similarity):
    #        i.remove(y[1])
    #    else:
    #      return_data_similarity = checkSimilarityDataTable(y[0],y[1])
    #      if (return_data_similarity):
    #        i.remove(y[0])
    '''
    for i in list_clean_parsing:
      for index,y in enumerate(i):
        i[index] = removeTitle(y)
    '''
    for i in list_clean_parsing:
      for index,y in enumerate(i):
        i[index] = checkMergedCell(y)

    return list_clean_parsing

    #return list_data     

Method parseTabel merupakan method untuk memprediksi tabel apa saja yang terdapat pada file excel tersebut. Library yang digunakan untuk memprediksi adalah skimage.measure

nilai kembaliannya berupa list yang berisi tabel-tabel setelah diparsing dalam bentuk data frame

In [ ]:
def parseTabel(data):
    data_array = np.array(data.T.reset_index().values.T.tolist())
    #binary_rep = np.array(data_array.notnull().astype('int'))
    data_array_copy = data_array.copy()
    size = data_array.shape

    for i in range(size[0]):
      for y in range(size[1]):
        if (str(data_array[i,y]) == 'nan' or 'Unnamed' in str(data_array[i,y])):
          data_array[i,y] = 0
        else:
          data_array[i,y] = 1

    list_of_dataframes = []
    l = label(data_array)
 
    for s in regionprops(l):
        slice_data = data_array_copy[s.bbox[0]:s.bbox[2],s.bbox[1]:s.bbox[3]]
        list_of_dataframes.append(pd.DataFrame(slice_data[1:], columns = slice_data[0]))
        #list_of_dataframes.append(data.iloc[s.bbox[0]:s.bbox[2],s.bbox[1]:s.bbox[3]])

    list_parseTable = []

    for i in list_of_dataframes:
        if (isItTable(i)):
            list_parseTable.append(i)
        else:
            continue

    return list_of_dataframes

Method isItTable merupakan method yang berfungsi untuk mengecek apakah tiap tabel hasil parsing pada method parsTabel merupakan benar-benar tabel atau bukan.

Logika yang digunakan untuk method ini adalah jika tabel tersebut memliki baris kurang dari atau sama dengan 1 maka otomatis bukan merupakan tabel. Lalu jika jika kondisi tersebut ternyata salah, maka kemudian akan dicek kembali. Jika baris dan kolom tabel tersebut kurang dari 2, maka otomatis bukan merupakan tabel

Nilai kembalian dari method ini adalah boolean, artinya jika setelah dicek ternyata bukan tabel, maka nilai kembaliannya adalah "False". Kemudian sebaliknya, jika setelah dicek ternyata merupakan tabel, maka nilai kembaliannya adalah "True"

In [ ]:
def isItTable(data):
    row,column = data.shape

    if (row <= 1):
        return False
    else:
        if (row <= 2 and column <= 2):
            return False
        else:
            return True

Method checkSimilarityDataTable dan similarityTable merupakan method untuk membandingkan antara dua tabel apakah sama atau tidak. Hal ini dilakukan untuk mengatasi masalah ketika menjalankan method parseTabel, karena bisa saja ketika melakukan parsing, sebuah tabel bisa saja dipisah menjadi 2 tabel

In [ ]:
def getIndexColumn(data):
  
  list_index_column = []
  for i in data:
    try:
       a = int(i)
       list_index_column.append(a)
    except:
      continue
  #print(list_index_column)
  result = 0

  if (len(list_index_column) <= 1):
    result = False
    #print('False 1')
  else:
    different_value = list_index_column[1] - list_index_column[0]

    for i in range(len(list_index_column)-1):
      #print(i)
      check = (i+1) - i
      if (check != different_value):
        result = False
        #print('False 2')
        break
      else:
        if ((i+1) == len(list_index_column)-1):
          if (list_index_column[0] >= 1000):
            result = False
          else:
            result = True
          #print('True')
  
  return result

def getNoneRow(data):
  list_noneData = []
  counter = 0
  
  for i in data:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      list_noneData.append(counter)
    counter += 1
  
  if (len(list_noneData) ==  len(data)):
    return True
  else:
    return False

def removeNoneRow(data):
  data_array = np.array(data.T.reset_index().values.T.tolist())
  data_result = []

  for i in range(len(data_array)):
    if (getNoneRow(data_array[i,:]) or getIndexColumn(data_array[i,:])):
      #print('halo')
      continue
    else:
      data_result.append(data_array[i,:])

  result = np.array(data_result)

  return pd.DataFrame(result[1:], columns = result[0])

In [ ]:
def checkSimilarityDataTable(table1, table2):
  np_array1 = np.array(table1.T.reset_index().values.T.tolist())
  np_array2 = np.array(table2.T.reset_index().values.T.tolist())

  rowSize1 = np_array1.shape[0]
  columnSize1 = np_array1.shape[1]
  rowSize2 = np_array2.shape[0]
  columnSize2 = np_array2.shape[1]
  
  number_true_value = 0
  counter = 0
  for i in range(rowSize1):
    for j in range(columnSize1):
      loc = np.where(np_array2 == np_array1[i,j])
      if (len(loc[0]) != 0):
        #print('True')
        counter += 1

  result = counter / (rowSize1*columnSize1)
  if (result == 1):
    return True
    #print('True')
  else: 
    return False
    #print('False')

def similarityTable(table1, table2):
  np_array1 = np.array(table1.T.reset_index().values.T.tolist())
  np_array2 = np.array(table2.T.reset_index().values.T.tolist())

  check1 = 0
  check2 = 0

  rowSize1 = np_array1.shape[0]
  columnSize1 = np_array1.shape[1]
  rowSize2 = np_array2.shape[0]
  columnSize2 = np_array2.shape[1]

  #===== check table1 in table2
  if (np_array2.shape[0] > np_array1.shape[0]):
    if (np_array1.shape[1] > np_array2.shape[1]):
      check1 = False
      #print("hai")
    else:
      startValue = np_array1[0,0]
      loc = np.where(np_array2 == startValue)
      for i in range(len(loc[0])):
        startRow = loc[0][i]
        startColumn = loc[1][i]
        selectedArray = np_array2[startRow:(startRow + rowSize1),startColumn:(startColumn + columnSize1)]
        rowSelectedArray = selectedArray.shape[0]
        columnSelectedArray = selectedArray.shape[1]        
        if ((rowSize1 == rowSelectedArray) and (columnSize1 == columnSelectedArray)):
        #print('True')
          check1 = True
        else:
        #print('False')
          check1 = False
  
  #=========== check table2 in table1
  if (np_array1.shape[0] > np_array2.shape[0]):
    if (np_array2.shape[1] > np_array1.shape[1]):
      check2 = False
    else:
      startValue = np_array2[0,0]
      loc = np.where(np_array1 == startValue)
      for i in range(len(loc[0])):
        startRow = loc[0][i]
        startColumn = loc[1][i]
        selectedArray = np_array1[startRow:(startRow + rowSize2),startColumn:(startColumn + columnSize2)]
        #print(selectedArray)
        rowSelectedArray = selectedArray.shape[0]
        columnSelectedArray = selectedArray.shape[1]        
        if ((rowSize2 == rowSelectedArray) and (columnSize2 == columnSelectedArray)):
        #print('True')
          check2 = True
        else:
        #print('False')
          check2 = False

  if (check1 == False):
    if (check2 == False):
      #print("nggak ada yang sama")
      return 1
    else:
      #print("tabel 2 masuk tabel 1")
      return 2
  else:
    #print("tabel 1 masuk tabel 2")
    return 3


Mehtod removeTitle merupakan method yang berfungsi untuk mendeteks judul yang ada pada tabel tersebut kemudian menghapusnya

In [75]:
def removeTitle(data):
  result = []
  data_array = np.array(data.T.reset_index().values.T.tolist())
  checkTitle, listIndex, data = 0, 0, 0

  counter = 0
  for i in range(len(data_array)):
    print(counter)
    checkTitle,listIndex = checkNoneData(data_array[i,:], (len(data_array[i,:])))

    if (checkTitle):
      if (i < (len(data_array)-1)):
        if (matchingRow(data_array[i,:], data_array[i+1,:])):
          if (checkTitle):
            continue
          else:
            result.append(data_array[i,:])
        else:
          if (i < (len(data_array)-2)):
            if (matchingRow3Layer(data_array[i,:], data_array[i+1,:], data_array[i+2,:])):
              if (checkTitle):
                continue
              else:
                result.append(data_array[i,:])
            else:
              continue
          else:
            continue
      else:
        continue
    else:
      result.append(data_array[i,:])
    counter += 1

  return pd.DataFrame(result[1:], columns = result[0])

def checkNoneData(listData, length):
  counter = 0
  list_index = []
  data = 0

  for i in listData:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      list_index.append(counter)
    counter += 1

  booleanResult = 0
  
  if ((length-1) in list_index and len(listData) > 3):
    if ((list_index[-1]-list_index[0]+1) == len(list_index)):
      booleanResult = True
    else:
      booleanResult = False
  else:
   booleanResult = False
  
  return booleanResult, list_index


Method checkMergeCell merupakan method yang berfungsi untuk menghapus adanya merge cell pada tabel

In [88]:
def checkMergedCell(data):
  data_array = np.array(data.T.reset_index().values.T.tolist())

  #print(matchingRow(data_array[0,:],data_array[1,:]))

  if (matchingRow(data_array[0,:],data_array[1,:])):
    #print(1)
    data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
    data_result = np.delete(data_array,1,0)
    return pd.DataFrame(data_result[1:], columns = data_result[0])

  elif (matchingRow3Layer(data_array[0,:],data_array[1,:], data_array[2,:])):
    #print(2)
    data_array[1,:] = renameHeader(data_array[1,:], data_array[2,:])
    data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
    data_result = np.delete(data_array,[1,2],0)
    return pd.DataFrame(data_result[1:], columns = data_result[0])
  elif (checkColumnMerge(data_array[0,:],data_array[1,:])[0]):
    index = checkColumnMerge(data_array[0,:],data_array[1,:])[1]
    counter = 1
    for i in index:
      data_array[0,i] = data_array[0,i-1] + "_" + str(counter)
      counter += 1
    
    data_array[0,:] = renameHeader(data_array[0,:], data_array[1,:])
    
    data_result = np.delete(data_array,1,0)
    return pd.DataFrame(data_result[1:], columns = data_result[0])
  else:
    #print(3)
    return pd.DataFrame(data_array[1:], columns = data_array[0])

def checkColumnMerge(data1, data2):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)

  list_index_noncover = []
  for i in None1:
    if (i in None2):
      list_index_noncover.append(i)
  
  index_merge = []
  for i in None1:
    if (i in NonNull2):
      continue
    else:
      index_merge.append(i)

  list_column_merge = []
  for i in index_merge:
    if (i in list_index_noncover):
      list_column_merge.append(i)

  if (len(list_column_merge) == 0):
    return False,0
  else:
    return True,list_column_merge

def matchingRow(data1, data2):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)
  #print(None1)
  #print(NonNull2)
  #print(None2)
  #print(NonNull1)
  check1 = checkIndex(None1, NonNull2)
  check2 = checkIndex(None2, NonNull1)
  if (check1 and check2):
    return True
  else:
    return False

#==== Tambahan 1.0 ========================================================
def matchingRow3Layer(data1, data2, data3):
  None1, NonNull1 = getNoneIndex(data1)
  None2, NonNull2 = getNoneIndex(data2)
  None3, NonNull3 = getNoneIndex(data3)
  
  list_null_cover = []
  list_null_nonCover = []
  counter = 0
  result = 0

  for i in None3:
    if (i in NonNull2):
      list_null_cover.append(counter)
    else:
      list_null_nonCover.append(counter)
    
    counter += 1
  
  listBoolean = []
  for i in NonNull1:
    if (i in NonNull2):  
      cek1 = 1
    else:
      cek1 = 2

    if (i in NonNull3):
      cek2 = 1
    else:
      cek2 = 2 

    if ((cek1 == cek2)):
      listBoolean.append(1)
    else:
      listBoolean.append(2)  
  
  if (2 in listBoolean):
    result = False
  else:
    result = True  
  
  if (len(None3) == 0):
    return False
  else:
    if (result):
      for i in None2:
        if (i in NonNull3):
          return True
        elif (i in list_null_nonCover):
          return True
        else:
          return False
    else:
    #print('False 2')
      return False
    
#================================================================================

def getNoneIndex(listData):
  counter = 0 
  listIndexNone = []
  listIndexNonNull = []
  for i in listData:
    if (str(i) == 'nan' or 'Unnamed' in str(i)):
      listIndexNone.append(counter)
    else:
      listIndexNonNull.append(counter)
    counter += 1
  return listIndexNone, listIndexNonNull

def checkIndex(dataNull, dataNonNull):
  for i in dataNull:
    if (i in dataNonNull):
      return True
    else:
      return False
      break

def renameHeader(data1, data2):
  indexNull1, indexNonNull1 = getNoneIndex(data1)
  indexNull2, indexNonNull2 = getNoneIndex(data2)

  duplicate_index = []
  data = data1.copy()

  for i in indexNonNull1:
    if (i in indexNonNull2):
      duplicate_index.append(i)

  for i in range(len(data1)):
    if (i in duplicate_index):
      data[i] = str(data2[i]) + "(" + str(data1[i]) + ")"
    elif ((i in indexNull1) and (i in indexNull2)):
      continue
    elif (i in indexNull1):
      index = findIndexMergedCell(duplicate_index, i)
      #print(index)
      data[i] = str(data2[i]) + "(" + str(data1[index]) + ")"
    else:
      continue
  
  return data


def findIndexMergedCell(duplicateIndex, index):

  for i in range(len(duplicateIndex)):
    if (index == duplicateIndex[i]):
      return index
      break
    else:
      if (i != (len(duplicateIndex)-1)):
        if (index > duplicateIndex[i] and index < duplicateIndex[i+1]):
          return duplicateIndex[i]
        else:
          continue
      else:
        if (index > duplicateIndex[i]):
          return duplicateIndex[i]

In [89]:
coba = cleaningTable('/content/DATA_DUMMY_MEDICAL_-_RS_-_KM_-_Januari_2020_s_d_November_2020.xlsx')
coba[0][0]

,NO,NRP,NAME BC,LOC.,LOC._1,OBAT(TAHUN 2020 (JAN,RS(TAHUN 2020 (JAN-N,KACAMATA(TAHUN 2020,TOTAL(TAHUN 2020 (JA
0,1,0101010,NAMA BC 1,HO,Dept 1,1000000,2000000,0,3000000
1,2,0101011,NAMA BC 2,HO,Dept 1,4000000,1500000,1500000,7000000
2,nan,0101014,NAMA BC 3,HO,Dept 2,5000000,3000000,1000000,9000000
3,nan,0101015,NAMA BC 4,Cab,Cab 1,6500000,0,0,6500000
4,nan,0101016,NAMA BC 5,Cab,Cab 1,4500000,25000000,500000,30000000


In [76]:
tes = removeTitle(coba[0][0])
tes

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
22
22
22


,Komoditas Ekspor,2016(Volume Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta),2015(Volume Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta),2014(Volume Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta),2016(Nilai Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta),2015(Nilai Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta),2014(Nilai Ekspor)(Ekspor menurut Mata Dagangan di D.I. Yogyakarta)
0,Pakaian Jadi/Tekstil,"2,18","3,95","4,02","48,55","56,89","49,77"
1,Sarung Tangan Kulit/,"0,68","0,56","1,10","31,11","26,37","35,56"
2,Mebel Kayu,"16,42","18,79","14,72","47,04","54,52","37,45"
3,STK Sintetis,"0,24","0,23","0,31","9,13","9,47","11,87"
4,Atsiri Daun Cengkeh,"1,73","1,20","0,38","27,41","15,25","7,29"
5,Kerajinan Kayu,"3,20","4,49","5,07","7,15","10,66","12,25"
6,Kerajinan Kertas,"0,90","1,03","1,11","4,81","3,77","2,62"
7,Minyak Kenanga,-,-,"0,05",-,-,"1,37"
8,Papan Kemas,"9,93","2,31","0,26","8,56","1,86","0,27"
9,Kulit Disamak,"0,09","0,20","1,38","5,52","9,21","6,16"


In [ ]:
#cek = cleaningTable("/content/DATA_DUMMY_Productivity_BC_as_of_30_Nov_2020.xlsx")
#cek[0][2]